<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#创建数据集" data-toc-modified-id="创建数据集-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>创建数据集</a></span><ul class="toc-item"><li><span><a href="#从数据数组创建数据集" data-toc-modified-id="从数据数组创建数据集-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>从数据数组创建数据集</a></span></li><li><span><a href="#读取文本文件里的数据" data-toc-modified-id="读取文本文件里的数据-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>读取文本文件里的数据</a></span></li><li><span><a href="#从TFRecord文件中获取数据" data-toc-modified-id="从TFRecord文件中获取数据-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>从TFRecord文件中获取数据</a></span></li><li><span><a href="#使用initializable_iterator来动态创建数据集" data-toc-modified-id="使用initializable_iterator来动态创建数据集-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>使用initializable_iterator来动态创建数据集</a></span></li></ul></li><li><span><a href="#参考资料" data-toc-modified-id="参考资料-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>参考资料</a></span></li></ul></div>

# 创建数据集

创建数据集的步骤一般如下:
    
1. 建立原始数据
2. 把原始数据转化为数据集
3. 建立迭代器
4. 获取数据

## 从数据数组创建数据集

In [1]:
import tempfile # 加载临时文件(夹)操作
import tensorflow as tf

In [ ]:
input_data = [1, 2, 3, 4, 5, 6, 7, 8]
dataset = tf.data.Dataset.from_tensor_slices(input_data)

# 定义迭代器
iterator = dataset.make_one_shot_iterator()

# get_next()返回代表一个输入数据的张量
x = iterator.get_next()
y = x * x

with tf.Session() as sess:
    for i in range(len(input_data)):
        print(sess.run(y), end=" ")

## 读取文本文件里的数据

In [ ]:
# 创建文本问津作为本例的输入
with open("../../../other/test1.txt", "w") as file:
    file.write("File1, line1.\n")
    file.write("File2, line2.\n")
with open("../../../other/test2.txt", "w") as file:
    file.write("File2, line1.\n")
    file.write("File2, line2.\n")
    
# 从文本文件创建数据集。这里看可以提供多个文件
input_files = ['../../../other/test1.txt', '../../../other/test2.txt']
dataset = tf.data.TextLineDataset(input_files)

# 定义迭代器
iterator = dataset.make_one_shot_iterator()

# 这里get_text()返回一个字符型类型的张量, 代表文件中的一行
x = iterator.get_next()
with tf.Session() as sess:
    for i in range(4):
        print(sess.run(x), end=" ")

## 从TFRecord文件中获取数据

In [2]:
# 解析一个TFRecord的方法
def parser(serialized_example):
    feature_i = tf.FixedLenFeature([], tf.int64)
    feature_j = tf.FixedLenFeature([], tf.int64)
    features_map = {'i': feature_i, "j": feature_j}
    features = tf.parse_single_example(serialized_example, features=features_map)
    
    example, label = features['i'], features['j']
    
    return example, label

# 从TFRecord文件创建数据集。这里可以提供多个文件
input_files = ["../../../other/test/data.tfrecords-00000-of-00010",
               "../../../other/test/data.tfrecords-00001-of-00010",
               "../../../other/test/data.tfrecords-00002-of-00010"
               "../../../other/test/data.tfrecords-00003-of-00010"
               "../../../other/test/data.tfrecords-00004-of-00010"
               "../../../other/test/data.tfrecords-00005-of-00010"]
dataset = tf.data.TFRecordDataset(input_files)

# map()函数表示对数据集中每一条数据进行调用解析方法
dataset = dataset.map(parser)

# 定义遍历数据集的迭代器
iterator = dataset.make_one_shot_iterator()

# 读取数据, 可用于进一步计算
example, label = iterator.get_next()

with tf.Session() as sess:
    for i in range(10):
        x, y =  sess.run([example, label])
        print(y, end=" ")

0 1 2 3 4 0 1 2 3 4 

## 使用initializable_iterator来动态创建数据集

In [3]:
# 从TFRecord文件创建数据集，具体文件路径是一个placeholer, 稍后提供具体地址
input_files = tf.placeholder(tf.string)
dataset = tf.data.TFRecordDataset(input_files)
dataset = dataset.map(parser)

# 定义遍历dataset的initializable_iterator
iterator = dataset.make_initializable_iterator()
example, label = iterator.get_next()

with tf.Session() as sess:
    # 首先初始化iterator, 并给出input_files的值
    sess.run(iterator.initializer, feed_dict={input_files: ["../../../other/test/data.tfrecords-00000-of-00010"]})
    
    # 遍历所有数据一个epoch.当遍历结束时, 程序会抛出OutOfRangeError
    while True:
        try:
            x, y = sess.run([example, label])
        except tf.errors.OutOfRangeError:
            break

# 参考资料

1. github